In [1]:
import numpy as np
import pandas as pd
from pyquery import PyQuery as pq
import time
import datetime
import requests
import urllib.request
import os
import calendar
import pickle
from lxml import etree

In [2]:
with open('search_addr_final.pkl','rb') as f:
    search_addr = pickle.load(f)

In [3]:
tickers = pd.read_csv('all_ticker.csv').iloc[:,1].values

In [4]:
with open('./10-Q_url/company_ft_dic_final.pkl','rb') as f:
    company_ft_dic = pickle.load(f)
# with open('./10-K_url/company_ft_dic_final.pkl','rb') as f:
#     company_ft_dic = pickle.load(f)

In [5]:
def crawl_fp_txt_url(company_url,fp_type = '10-Q'):
    finance_report_url = []
    finance_datetime = [] # 發佈時間
    pop_datetime = [] # 實際時間
    mark = []
    res = requests.get(company_url)
    doc = pq(res.text)
    df_list = pd.read_html(company_url)
    
    skip_index = []
    
    # There are tables in the url
    if(len(df_list) >= 3):
        df = df_list[2]
        df_time = df.iloc[:,3].values
        for i,ty in enumerate(df.iloc[:,0].values):
            if(ty == fp_type):
                finance_datetime.append(datetime.datetime.strptime(df_time[i], '%Y-%m-%d'))
                pass
            else:
                skip_index.append(i)
    
    temp = 'https://www.sec.gov'

    for i,d in enumerate(doc('.tableFile2 #documentsbutton').items()):
#         print(d)
        if(not i in skip_index):
            f_url = temp+d.attr('href')    
            lv1_res = requests.get(f_url)
            lv1_doc = pq(lv1_res.text)
            txt_num = lv1_doc('#secNum').text().split(' ')[-1]
            
            for d_ in lv1_doc('.formGrouping').items():
                if(d_('.infoHead').text() == 'Period of Report'):
                    ddd = d_('.info').text()
            pop_datetime.append(datetime.datetime.strptime(ddd, '%Y-%m-%d'))
            
            htm_pd = pd.read_html(f_url)
            htm_num_ = htm_pd[0].iloc[0,2]

            sel = etree.HTML(lv1_res.text)
            
            try:
                if (type(htm_num_) == str) :
                    htm_num = htm_num_.split(' ')[0]
                    url = sel.xpath(f"//a[contains(text(),'{htm_num}')] ")[0].attrib['href']
                    mark.append(0)
                else:
                    url = sel.xpath(f"//a[contains(text(),'{txt_num}.txt')] ")[0].attrib['href']
                    mark.append(1)
            except:
                print('Company',company_url)
                print(f_url)
                print(htm_num)
                print(df_time[i])
                print(txt_num)
            finance_report_url.append(url)
        
    if (len(finance_report_url) == 0):
        finance_report_url = 'Unfounded'
        
    return finance_report_url, finance_datetime, pop_datetime, mark

In [82]:
finance_report_url, finance_datetime, pop_datetime, mark = crawl_fp_txt_url('https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001642453&type=10-K&dateb=&owner=exclude&count=40','10-K')

<a href="/Archives/edgar/data/1642453/000155837019003486/0001558370-19-003486-index.htm" id="documentsbutton"> Documents</a>
<a href="/Archives/edgar/data/1642453/000155837019001728/0001558370-19-001728-index.htm" id="documentsbutton"> Documents</a>  
<a href="/Archives/edgar/data/1642453/000155837018002217/0001558370-18-002217-index.htm" id="documentsbutton"> Documents</a>  
<a href="/Archives/edgar/data/1642453/000121390017001766/0001213900-17-001766-index.htm" id="documentsbutton"> Documents</a>  
<a href="/Archives/edgar/data/1642453/000121390016011152/0001213900-16-011152-index.htm" id="documentsbutton"> Documents</a>  
<a href="/Archives/edgar/data/1642453/000121390016011096/0001213900-16-011096-index.htm" id="documentsbutton"> Documents</a>  


In [83]:
finance_report_url

['/Archives/edgar/data/1642453/000155837019001728/dske-20181231x10k.htm',
 '/Archives/edgar/data/1642453/000155837018002217/dske-20171231x10k.htm',
 '/Archives/edgar/data/1642453/000121390017001766/f10k2016_hennessycapital.htm',
 '/Archives/edgar/data/1642453/000121390016011096/f10k2015_hennessycapital.htm']

In [84]:
finance_datetime

[datetime.datetime(2019, 3, 8, 0, 0),
 datetime.datetime(2018, 3, 16, 0, 0),
 datetime.datetime(2017, 2, 24, 0, 0),
 datetime.datetime(2016, 2, 19, 0, 0)]

In [85]:
pop_datetime

[datetime.datetime(2018, 12, 31, 0, 0),
 datetime.datetime(2017, 12, 31, 0, 0),
 datetime.datetime(2016, 12, 31, 0, 0),
 datetime.datetime(2015, 12, 31, 0, 0)]

In [58]:
res = requests.get('https://www.sec.gov/Archives/edgar/data/706539/0001047469-98-012226-index.html')
doc = pq(res.text)

In [76]:
for i,d in enumerate(doc('.formGrouping').items()):
    if(d('.infoHead').text() == 'Period of Report'):
        print(d('.info').text())

1997-12-28


In [8]:
def get_all_fp_url(tickers,company_addr):
    # check directory
    if not os.path.isdir('./fp_url'):
        os.mkdir('./fp_url')
    
    # check files
    files = os.listdir('./fp_url')
    max_ver = 0

    for f in files:
        if 'company' in f:
            version = int(f.split('_')[3].split('.')[0])
            if (version > max_ver ):
                max_ver = version
                max_f = f
    if (max_ver == 0):     
        company_fp_dic = {}
    else:
        if (os.path.getsize(f"./fp_url/{max_f}") > 0):
            with open(f'./fp_url/{max_f}','rb') as f:
                company_fp_dic = pickle.load(f)
        else:
            company_fp_dic = {}
    length = len(company_fp_dic)   
    
    for i,key in enumerate(tickers[length:]):
        addr = company_addr[key]
        
        if (addr != 'Unfounded'):
            fp_addr = crawl_fp_txt_url(addr)
            company_fp_dic[key] = fp_addr
        else:
            company_fp_dic[key] = 'Unfounded'
            
        if(len(company_fp_dic)%1000 == 0):
            num = len(company_fp_dic)/1000

            with open(f"./fp_url/company_fp_dic_{num}.pkl","wb") as f:
                pickle.dump(company_fp_dic,f)
                print(f'./fp_url/company_fp_dic_{num}.pkl',i+length)
                
    with open(f"./fp_url/company_fp_dic_final.pkl","wb") as f:
        pickle.dump(company_fp_dic,f)
        print(f'./fp_url/company_fp_dic_final.pkl',i+length)

In [ ]:
get_all_fp_url(tickers,company_ft_dic)

./fp_url/company_fp_dic_1.0.pkl 999


In [10]:
def get_all_fp_url_htm(tickers,company_addr,fp_type = '10-Q'):
    # check directory
    if not os.path.isdir(f'./{fp_type}'):
        os.mkdir(f'./{fp_type}')
    if not os.path.isdir(f'./{fp_type}/fp_url_htm'):
        os.mkdir(f'./{fp_type}/fp_url_htm')
    
    # check files
    files = os.listdir(f'./{fp_type}/fp_url_htm')
    max_ver = 0

    for f in files:
        if 'company_pop' in f:
            version = int(f.split('_')[3].split('.')[0])
            if (version > max_ver ):
                max_ver = version
                max_fp_f = f.replace('pop','fp')
                max_dt_f = f.replace('fp','dt')
                max_mk_f = f.replace('fp','mk')
                max_pop_f = f
    if (max_ver == 0):     
        company_fp_dic = {}
        company_dt_dic = {}
        company_mk_dic = {}
        company_pop_dic = {}
    else:
        if (os.path.getsize(f"./{fp_type}/fp_url_htm/{max_fp_f}") > 0):
            with open(f'./{fp_type}/fp_url_htm/{max_fp_f}','rb') as f:
                company_fp_dic = pickle.load(f)
        else:
            company_fp_dic = {}
            
        if (os.path.getsize(f"./{fp_type}/fp_url_htm/{max_dt_f}") > 0):
            with open(f'./{fp_type}/fp_url_htm/{max_dt_f}','rb') as f:
                company_dt_dic = pickle.load(f)
        else:
            company_dt_dic = {} 
        
        if (os.path.getsize(f"./{fp_type}/fp_url_htm/{max_mk_f}") > 0):
            with open(f'./{fp_type}/fp_url_htm/{max_mk_f}','rb') as f:
                company_mk_dic = pickle.load(f)
        else:
            company_mk_dic = {} 
            
        if (os.path.getsize(f"./{fp_type}/fp_url_htm/{max_pop_f}") > 0):
            with open(f'./{fp_type}/fp_url_htm/{max_pop_f}','rb') as f:
                company_pop_dic = pickle.load(f)
        else:
            company_pop_dic = {} 
            
    length = len(company_pop_dic)   
    
    for i,key in enumerate(tickers[length:]):
#         print(key)
        addr = company_addr[key]
        
        if (addr != 'Unfounded'):
            fp_addr, fp_datetime, fp_pop, fp_mark = crawl_fp_txt_url(addr,fp_type)
            
            company_fp_dic[key] = fp_addr
            company_dt_dic[key] = fp_datetime
            company_mk_dic[key] = fp_mark
            company_pop_dic[key] = fp_pop
        else:
            company_fp_dic[key] = 'Unfounded'
            company_dt_dic[key] = 'Unfounded'
            company_mk_dic[key] = 'Unfounded'
            company_pop_dic[key] = 'Unfounded'
            
        if(len(company_pop_dic)%1000 == 0):
            num = len(company_pop_dic)/1000

            with open(f"./{fp_type}/fp_url_htm/company_fp_dic_{num}.pkl","wb") as f:
                pickle.dump(company_fp_dic,f)
                print(f'./{fp_type}/fp_url_htm/company_fp_dic_{num}.pkl',i+length)
                
            with open(f"./{fp_type}/fp_url_htm/company_dt_dic_{num}.pkl","wb") as f:
                pickle.dump(company_dt_dic,f)
                print(f'./{fp_type}/fp_url_htm/company_dt_dic_{num}.pkl',i+length)
            
            with open(f"./{fp_type}/fp_url_htm/company_mk_dic_{num}.pkl","wb") as f:
                pickle.dump(company_mk_dic,f)
                
            with open(f"./{fp_type}/fp_url_htm/company_pop_dic_{num}.pkl","wb") as f:
                pickle.dump(company_pop_dic,f)
                
    with open(f"./{fp_type}/fp_url_htm/company_fp_dic_final.pkl","wb") as f:
        pickle.dump(company_fp_dic,f)
        print(f'./{fp_type}/fp_url_htm/company_fp_dic_final.pkl',i+length)
        
    with open(f"./{fp_type}/fp_url_htm/company_dt_dic_final.pkl","wb") as f:
        pickle.dump(company_dt_dic,f)
        print(f'./{fp_type}/fp_url_htm/company_dt_dic_final.pkl',i+length)
        
    with open(f"./{fp_type}/fp_url_htm/company_mk_dic_final.pkl","wb") as f:
        pickle.dump(company_mk_dic,f)
        
    with open(f"./{fp_type}/fp_url_htm/company_pop_dic_final.pkl","wb") as f:
        pickle.dump(company_pop_dic,f)

In [ ]:
get_all_fp_url_htm(tickers,company_ft_dic,'10-Q')

In [8]:
company_ft_dic

{'FSLA': 'Unfounded',
 'DSKE': 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001642453&type=10-Q&dateb=&owner=exclude&count=100',
 'CHIR': 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000706539&type=10-Q&dateb=&owner=exclude&count=100',
 'NIV': 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001403795&type=10-Q&dateb=&owner=exclude&count=100',
 'TNO': 'Unfounded',
 'GWGH': 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001522690&type=10-Q&dateb=&owner=exclude&count=100',
 'IV': 'Unfounded',
 'MERI': 'Unfounded',
 'PLYM': 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001515816&type=10-Q&dateb=&owner=exclude&count=100',
 'MERR': 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000826683&type=10-Q&dateb=&owner=exclude&count=100',
 'TIK': 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000096885&type=10-Q&dateb=&owner=exclude&count=100',
 'WNNB': 'Unfounded',
 'OXPS

In [40]:
fp_type = '10-K'

In [41]:
# datetime of each financial report
with open(f'./{fp_type}/fp_url_htm/company_dt_dic_final.pkl','rb') as f:
    company_dt_dic = pickle.load(f)
    
# htm or txt url of each financial report
with open(f'./{fp_type}/fp_url_htm/company_fp_dic_final.pkl','rb') as f:
    company_fp_dic = pickle.load(f)
    
# mark of each financial report
# 0 : crawl from raw 1(htm or txt)  1: crawl from original txt
with open(f'./{fp_type}/fp_url_htm/company_mk_dic_final.pkl','rb') as f:
    company_mk_dic = pickle.load(f)

In [42]:
company_dt_dic

{'FSLA': 'Unfounded',
 'DSKE': [datetime.datetime(2019, 3, 8, 0, 0),
  datetime.datetime(2018, 3, 16, 0, 0),
  datetime.datetime(2017, 2, 24, 0, 0),
  datetime.datetime(2016, 2, 19, 0, 0)],
 'CHIR': [datetime.datetime(2006, 3, 16, 0, 0),
  datetime.datetime(2005, 3, 16, 0, 0),
  datetime.datetime(2004, 3, 3, 0, 0),
  datetime.datetime(2003, 3, 5, 0, 0),
  datetime.datetime(2002, 3, 6, 0, 0),
  datetime.datetime(2001, 3, 7, 0, 0),
  datetime.datetime(2000, 3, 10, 0, 0),
  datetime.datetime(1999, 3, 16, 0, 0),
  datetime.datetime(1998, 3, 30, 0, 0),
  datetime.datetime(1997, 3, 31, 0, 0),
  datetime.datetime(1996, 3, 28, 0, 0),
  datetime.datetime(1995, 3, 17, 0, 0)],
 'NIV': [datetime.datetime(2010, 3, 24, 0, 0),
  datetime.datetime(2009, 3, 31, 0, 0)],
 'TNO': 'Unfounded',
 'GWGH': [datetime.datetime(2018, 3, 29, 0, 0),
  datetime.datetime(2017, 3, 15, 0, 0),
  datetime.datetime(2016, 3, 22, 0, 0),
  datetime.datetime(2015, 3, 13, 0, 0),
  datetime.datetime(2014, 3, 20, 0, 0),
  dateti

In [43]:
company_fp_dic

{'FSLA': 'Unfounded',
 'DSKE': ['/Archives/edgar/data/1642453/000155837019001728/dske-20181231x10k.htm',
  '/Archives/edgar/data/1642453/000155837018002217/dske-20171231x10k.htm',
  '/Archives/edgar/data/1642453/000121390017001766/f10k2016_hennessycapital.htm',
  '/Archives/edgar/data/1642453/000121390016011096/f10k2015_hennessycapital.htm'],
 'CHIR': ['/Archives/edgar/data/706539/000110465906017228/a06-1853_110k.htm',
  '/Archives/edgar/data/706539/000110465905011453/a05-3466_110k.htm',
  '/Archives/edgar/data/706539/000104746904006411/a2129339z10-k.htm',
  '/Archives/edgar/data/706539/000104746903007803/a2104185z10-k.htm',
  '/Archives/edgar/data/706539/000091205702008776/a2071810z10-k.htm',
  '/Archives/edgar/data/706539/000091205701007334/a2039414z10-k.txt',
  '/Archives/edgar/data/706539/000091205700010740/0000912057-00-010740.txt',
  '/Archives/edgar/data/706539/0001047469-99-009828.txt',
  '/Archives/edgar/data/706539/0001047469-98-012226.txt',
  '/Archives/edgar/data/706539/000


### Some test


In [44]:
fp_type = '10-K'

In [1]:
def check_index(company_fp_dic,company_dt_dic,tickers):
    '''
    Check alignment for url and datetime.
    '''
    errors = []
    for tk in tickers:
        dt_ele = company_dt_dic[tk]
        fp_ele = company_fp_dic[tk]
        
        if(company_dt_dic[tk] == 'Unfounded'):
            try:
                assert fp_ele == dt_ele
            except:
                print(tk)
                errors.append(tk)
        else:
            try:
                assert len(fp_ele) == len(dt_ele)
            except:
                print(tk)
                errors.append(tk)
                
    assert errors == []

In [46]:
for key,value in company_dt_dic.items():
    if value == []:
        company_dt_dic[key] = 'Unfounded'

In [47]:
with open(f'./{fp_type}/fp_url_htm/company_dt_dic_final.pkl','wb') as f:
    pickle.dump(company_dt_dic,f)

In [48]:
def remove_repeat_ones(company_dt_dic,company_fp_dic,company_mk_dic,tickers):
    for tk in tickers:
        if(company_dt_dic[tk] != 'Unfounded'):
#             print(tk)
            dts = np.array(company_dt_dic[tk])
            fps = np.array(company_fp_dic[tk])
            mks = np.array(company_mk_dic[tk])
        
            length = dts.shape[0]
            repeat_indexs = []
            temp = dts.copy()

            for i in range(length-1):
                if(temp[i] == temp[i+1]):
                    repeat_indexs.append(i+1)

            company_dt_dic[tk] = np.delete(dts,repeat_indexs).tolist()
            company_fp_dic[tk] = np.delete(fps,repeat_indexs).tolist()
            company_mk_dic[tk] = np.delete(mks,repeat_indexs).tolist()
        

In [49]:
def check_repeat(company_dt_dic,tickers):
    for tk in tickers:
        if(company_dt_dic[tk] != 'Unfounded'):
            dts = np.array(company_dt_dic[tk])
            fps = np.array(company_fp_dic[tk])
            mks = np.array(company_mk_dic[tk])
        
            length = dts.shape[0]
            repeat_indexs = []
            temp = dts.copy()

            for i in range(length-1):
                if(temp[i] == temp[i+1]):
                    repeat_indexs.append(i+1)
            assert repeat_indexs == []
    print(True)

In [50]:
remove_repeat_ones(company_dt_dic,company_fp_dic,company_mk_dic,tickers)

In [51]:
check_index(company_fp_dic,company_dt_dic,tickers)

[]

In [52]:
check_repeat(company_dt_dic,tickers)

True


In [179]:
with open(f"./{fp_type}/fp_url_htm/company_fp_dic_final.pkl","wb") as f:
    pickle.dump(company_fp_dic,f)
        
with open(f"./{fp_type}/fp_url_htm/company_dt_dic_final.pkl","wb") as f:
    pickle.dump(company_dt_dic,f)

with open(f"./{fp_type}/fp_url_htm/company_mk_dic_final.pkl","wb") as f:
    pickle.dump(company_mk_dic,f)

In [53]:
f"./{fp_type}/fp_url_htm/company_fp_dic_final.pkl"

'./10-K/fp_url_htm/company_fp_dic_final.pkl'

In [54]:
f"./{fp_type}/fp_url_htm/company_dt_dic_final.pkl"

'./10-K/fp_url_htm/company_dt_dic_final.pkl'